<a href="https://colab.research.google.com/github/MarianoChic09/Agentes-Inteligentes/blob/main/MSc_AI_ORT_Taller_IA_Tarea1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller de Deep Learning - Tarea 1

Introducción:
El objetivo de esta tarea es evaluar su conocimiento sobre Deep Learning aplicado a un caso de uso real. En particular, vamos a evaluar la performance de sus modelos en una tarea de clasificación de escenas.

Dataset:
El dataset a ser utilizado consiste de más de 13.000 imágenes de entrenamiento agrupadas
en 6 posibles “escenarios”: edificios, bosques, glaciares, montañas, océanos, calles.
Pueden descargarlo en el siguiente link.
El mismo contiene dos carpetas (train_set y test_set) donde las imágenes están
agrupadas en sub carpetas de acuerdo a su categoría.

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
! cp -r "/content/drive/MyDrive/Colab Notebooks/Datasets/scene_classification" .
! unzip -q scene_classification.zip
! rm scene_classification.zip

unzip:  cannot find or open scene_classification.zip, scene_classification.zip.zip or scene_classification.zip.ZIP.
rm: cannot remove 'scene_classification.zip': No such file or directory


In [3]:
#%cd "/content/drive/MyDrive/Colab Notebooks/Datasets/scene_classification"
#%ls

/content/drive/MyDrive/Colab Notebooks/Datasets/scene_classification
test_set/  train_set/


## Cargando el dataset

In [7]:
import os

print("Directorio actual:", os.getcwd())
print("\nContenido de 'train_set':", os.listdir("train_set"))

Directorio actual: /content

Contenido de 'train_set': ['street', 'glacier', 'sea', 'mountain', 'buildings', 'forest']


In [8]:
# Reemplaza 'class_1' con el nombre de uno de tus subdirectorios.
print("Contenido de 'train_set/buildings':", os.listdir("train_set/buildings"))

Contenido de 'train_set/buildings': ['18565.jpg', '12649.jpg', '12432.jpg', '8824.jpg', '17154.jpg', '11605.jpg', '12135.jpg', '1503.jpg', '18240.jpg', '16607.jpg', '2043.jpg', '8723.jpg', '11929.jpg', '2809.jpg', '6265.jpg', '9498.jpg', '15098.jpg', '13352.jpg', '5788.jpg', '18991.jpg', '7563.jpg', '3253.jpg', '13901.jpg', '19513.jpg', '2607.jpg', '12818.jpg', '17969.jpg', '801.jpg', '11349.jpg', '11658.jpg', '11181.jpg', '13477.jpg', '8418.jpg', '4359.jpg', '12273.jpg', '15784.jpg', '9127.jpg', '16681.jpg', '14896.jpg', '17296.jpg', '12068.jpg', '8100.jpg', '7610.jpg', '4517.jpg', '8380.jpg', '11197.jpg', '9202.jpg', '16543.jpg', '6266.jpg', '10626.jpg', '8964.jpg', '1415.jpg', '8966.jpg', '10350.jpg', '5798.jpg', '12171.jpg', '13882.jpg', '18962.jpg', '8701.jpg', '18887.jpg', '15078.jpg', '2430.jpg', '8107.jpg', '10620.jpg', '14318.jpg', '18655.jpg', '6075.jpg', '18157.jpg', '7710.jpg', '8659.jpg', '18050.jpg', '9716.jpg', '15176.jpg', '2069.jpg', '617.jpg', '17662.jpg', '17264.jpg'

In [9]:
all_extensions = set()  # Para almacenar todas las extensiones encontradas

# Iterar sobre cada subdirectorio en "train_set"
for folder in os.listdir("train_set"):
    # Determinar las extensiones de archivos en el subdirectorio actual
    extensions = {os.path.splitext(file)[1].lower() for file in os.listdir(f"train_set/{folder}")}
    all_extensions.update(extensions)

    print(f"Para la carpeta {folder}:")
    print(extensions)
    print("-" * 50)

print("Todas las extensiones encontradas:", all_extensions)

Para la carpeta street:
{'.jpg'}
--------------------------------------------------
Para la carpeta glacier:
{'.jpg'}
--------------------------------------------------
Para la carpeta sea:
{'.jpg'}
--------------------------------------------------
Para la carpeta mountain:
{'.jpg'}
--------------------------------------------------
Para la carpeta buildings:
{'.jpg'}
--------------------------------------------------
Para la carpeta forest:
{'.jpg'}
--------------------------------------------------
Todas las extensiones encontradas: {'.jpg'}


In [12]:
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from sklearn.metrics import accuracy_score, confusion_matrix
import time
import torch

class CustomCNN(nn.Module):
  def __init__(self, in_channels,number_classes):
    # in_channels: int, cantidad de canales de la imagen original
    super(CustomCNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels, out_channels = 32, kernel_size = 3,padding=1)
    self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3,padding=1)
    self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3,padding=1)
    self.conv4 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3,padding=1)
    self.conv5 = nn.Conv2d(in_channels = 64, out_channels = 120, kernel_size = 3,padding=1)

    self.linear1 = nn.Linear(in_features = 120*18*18, out_features = 512)
    self.linear2 = nn.Linear(in_features = 512, out_features = number_classes)

    self.max_pooling = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.dropout = nn.Dropout(p=0.5)

  def forward(self, x):
    out = F.relu(self.conv1(x))
    out = F.relu(self.conv2(out))
    out = self.max_pooling(out)

    out = F.relu(self.conv3(out))
    out = F.relu(self.conv4(out))
    out = self.max_pooling(out)

    out = F.relu(self.conv5(out))
    out = self.max_pooling(out)

    out = self.dropout(out).flatten(1)

    out = F.relu(self.linear1(out))
    out = self.dropout(out)
    out = self.linear2(out)
    return out

In [13]:
# def get_dataloaders(train_dataset, test_dataset, batch_size):

#     train_size = int(0.8 * len(train_dataset))
#     valid_size = len(train_dataset) - train_size
#     train_dataset, validation_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size])

#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
#     valid_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
#     test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)

#     return train_loader, valid_loader, test_loader

def train_epoch(training_model, loader, criterion, optim):
    training_model.train()
    epoch_loss = 0.0
    all_labels = []
    all_predictions = []

    for images, labels in loader:
      all_labels.extend(labels.numpy())

      optim.zero_grad()

      predictions = training_model(images.to(device))
      all_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())

      loss = criterion(predictions, labels.to(device))

      loss.backward()
      optim.step()

      epoch_loss += loss.item()

    return epoch_loss / len(loader), accuracy_score(all_labels, all_predictions) * 100


def validation_epoch(val_model, loader, criterion):
    val_model.eval()
    epoch_loss = 0.0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
      for images, labels in loader:
        all_labels.extend(labels.numpy())

        predictions = val_model(images.to(device))
        all_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())

        loss = criterion(predictions, labels.to(device))

        epoch_loss += loss.item()

    return epoch_loss / len(loader), accuracy_score(all_labels, all_predictions) * 100


def train_model(model, train_loader, test_loader, criterion, optim, number_epochs):
  train_history = []
  test_history = []
  accuracy_history = []

  for epoch in range(number_epochs):
      start_time = time.time()

      train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
      train_history.append(train_loss)
      print("Training epoch {} | Loss {:.6f} | Accuracy {:.2f}% | Time {:.2f} seconds"
            .format(epoch + 1, train_loss, train_acc, time.time() - start_time))

      start_time = time.time()
      test_loss, acc = validation_epoch(model, test_loader, criterion)
      test_history.append(test_loss)
      accuracy_history.append(acc)
      print("Validation epoch {} | Loss {:.6f} | Accuracy {:.2f}% | Time {:.2f} seconds"
            .format(epoch + 1, test_loss, acc, time.time() - start_time))

def test_model(test_model, loader, criterion):
    test_model.eval()
    epoch_loss = 0.0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in loader:
            all_labels.extend(labels.numpy())

            predictions = test_model(images.to(device))
            all_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())

            loss = criterion(predictions, labels.to(device))
            epoch_loss += loss.item()

    return epoch_loss / len(loader), accuracy_score(all_labels, all_predictions) * 100

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [14]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# 1. Setup the transformations and datasets
# Set random seed for reproducibility
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define your transformations
train_transform = transforms.Compose([
    transforms.Resize((150, 150)),  # Resizing to ensure all images are the same size
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((150, 150)),  # Resizing to ensure all images are the same size
    transforms.ToTensor()
])

# Define your datasets using the transformations
train_dataset = ImageFolder("train_set", transform=train_transform)
test_dataset = ImageFolder("test_set", transform=test_transform)

# 2. Create dataloaders using your dataset splits
BATCH_SIZE = 10

# Split the train_dataset into train and validation
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size
train, validation = torch.utils.data.random_split(train_dataset, [train_size, valid_size])
print(f"{len(train)} Training images, {len(validation)} Validation images")

# Create your dataloaders
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
val_loader = DataLoader(validation, batch_size=BATCH_SIZE, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, pin_memory=True)

#
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# Global models config

BATCH_SIZE = 10
LR = 0.001
NUMBER_EPOCHS = 15
criterion = nn.CrossEntropyLoss().to(device)

# 3. Define and train your model
# Assuming you've already defined CustomCNN, criterion, and other functions earlier.
modelo_sin_aug = CustomCNN(3, 10).to(device)
optimizer = torch.optim.Adam(modelo_sin_aug.parameters(), lr=LR)

# Now, you can train your model
train_model(modelo_sin_aug, train_loader, val_loader, criterion, optimizer, NUMBER_EPOCHS)


11227 Training images, 2807 Validation images
cuda:0
Training epoch 1 | Loss 1.060411 | Accuracy 57.15% | Time 48.11 seconds
Validation epoch 1 | Loss 0.834662 | Accuracy 67.22% | Time 3.82 seconds
Training epoch 2 | Loss 0.771453 | Accuracy 70.27% | Time 40.22 seconds
Validation epoch 2 | Loss 0.640809 | Accuracy 76.34% | Time 3.79 seconds
Training epoch 3 | Loss 0.654437 | Accuracy 75.42% | Time 39.04 seconds
Validation epoch 3 | Loss 0.689604 | Accuracy 75.67% | Time 3.70 seconds
Training epoch 4 | Loss 0.573995 | Accuracy 78.97% | Time 39.31 seconds
Validation epoch 4 | Loss 0.587456 | Accuracy 78.48% | Time 3.80 seconds
Training epoch 5 | Loss 0.491954 | Accuracy 82.05% | Time 39.29 seconds
Validation epoch 5 | Loss 0.532517 | Accuracy 81.40% | Time 3.79 seconds
Training epoch 6 | Loss 0.434489 | Accuracy 84.35% | Time 39.15 seconds
Validation epoch 6 | Loss 0.561322 | Accuracy 80.41% | Time 4.19 seconds
Training epoch 7 | Loss 0.374195 | Accuracy 86.91% | Time 39.02 seconds
Valid

--------

In [16]:
test_loss, test_accuracy = test_model(modelo_sin_aug, test_loader, criterion)
print("Test Loss: {:.6f}".format(test_loss))
print("Test Accuracy: {:.2f}%".format(test_accuracy))


Test Loss: 0.662607
Test Accuracy: 81.40%
